In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('./data/ADAUSDT_5m.csv')
df.head()

,time,open,high,low,close,volume
0,2023-08-01 00:00:00,0.01835,0.01835,0.01831,0.01833,3294190.0
1,2023-08-01 00:05:00,0.01832,0.01832,0.01823,0.01827,7665241.0
2,2023-08-01 00:10:00,0.01828,0.01829,0.01824,0.01825,2194684.0
3,2023-08-01 00:15:00,0.01826,0.01829,0.01824,0.01828,3216354.0
4,2023-08-01 00:20:00,0.01827,0.01830,0.01827,0.01827,978311.0


In [2]:
import pandas_ta as ta

temp = df.ta.bbands()
df['BBANDS_U'] = temp['BBU_5_2.0']
df['BBANDS_L'] = temp['BBL_5_2.0']
df['BBANDS_M'] = temp['BBM_5_2.0']
temp = df.ta.cdl_pattern(name="closingmarubozu")
df['MARUBOZU'] = temp['CDL_CLOSINGMARUBOZU']
#singal_buy = MARUBOZU == 100 and close > DONCH_U 
#singal_sell = MARUBOZU == -100 and close < DONCH_L
df['Signal_Buy'] = np.where((df['MARUBOZU'] == 100) & (df['close'] < df['BBANDS_U']), 1, 0)
df['Signal_Sell'] = np.where((df['MARUBOZU'] == -100) & (df['close'] < df['BBANDS_L']), 1, 0)
df['Signal'] = df['Signal_Buy'].replace(1,2) + df['Signal_Sell']
df['ATR'] = df.ta.atr()
df['ADX'] = df.ta.adx()['ADX_14']
#help(ta.adx)

In [3]:
df['Signal'].value_counts()

0    21075
2     2416
1        9
Name: Signal, dtype: int64

In [4]:
df.ta.bbands()

,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,0.018226,0.018280,0.018334,0.587151,0.406831
...,...,...,...,...,...
23495,0.013558,0.013574,0.013590,0.235745,0.687500
23496,0.013552,0.013570,0.013588,0.263648,0.220492
23497,0.013552,0.013570,0.013588,0.263648,0.779508
23498,0.013552,0.013570,0.013588,0.263648,0.500000


In [5]:
help(ta.donchian)

Help on function donchian in module pandas_ta.volatility.donchian:

donchian(high, low, lower_length=None, upper_length=None, offset=None, **kwargs)
    Donchian Channels (DC)
    
    Donchian Channels are used to measure volatility, similar to
    Bollinger Bands and Keltner Channels.
    
    Sources:
        https://www.tradingview.com/wiki/Donchian_Channels_(DC)
    
    Calculation:
        Default Inputs:
            lower_length=upper_length=20
        LOWER = low.rolling(lower_length).min()
        UPPER = high.rolling(upper_length).max()
        MID = 0.5 * (LOWER + UPPER)
    
    Args:
        high (pd.Series): Series of 'high's
        low (pd.Series): Series of 'low's
        lower_length (int): The short period. Default: 20
        upper_length (int): The short period. Default: 20
        offset (int): How many periods to offset the result. Default: 0
    
    Kwargs:
        fillna (value, optional): pd.DataFrame.fillna(value)
        fill_method (value, optional): Type

In [6]:
#dropna
df = df.dropna()
df.head()

,time,open,high,low,close,volume,BBANDS_U,BBANDS_L,BBANDS_M,MARUBOZU,Signal_Buy,Signal_Sell,Signal,ATR,ADX
27,2023-08-01 02:15:00,0.01791,0.01798,0.01788,0.01796,10706295.0,0.018289,0.017791,0.018040,0.0,0,0,0,0.000093,45.202645
28,2023-08-01 02:20:00,0.01796,0.01798,0.01791,0.01791,3897690.0,0.018214,0.017762,0.017988,0.0,0,0,0,0.000091,48.558813
29,2023-08-01 02:25:00,0.01791,0.01796,0.01790,0.01794,3066078.0,0.017975,0.017893,0.017934,0.0,0,0,0,0.000089,51.489473
30,2023-08-01 02:30:00,0.01793,0.01797,0.01790,0.01792,4510482.0,0.017967,0.017889,0.017928,0.0,0,0,0,0.000088,53.778292
31,2023-08-01 02:35:00,0.01793,0.01798,0.01793,0.01798,2124079.0,0.017993,0.017891,0.017942,0.0,0,0,0,0.000086,55.517775


In [7]:
#set time to index

df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')
df.head()

C:\Users\hiban\AppData\Local\Temp\ipykernel_10900\1852176384.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time'] = pd.to_datetime(df['time'])


,open,high,low,close,volume,BBANDS_U,BBANDS_L,BBANDS_M,MARUBOZU,Signal_Buy,Signal_Sell,Signal,ATR,ADX
time,,,,,,,,,,,,,,
2023-08-01 02:15:00,0.01791,0.01798,0.01788,0.01796,10706295.0,0.018289,0.017791,0.018040,0.0,0,0,0,0.000093,45.202645
2023-08-01 02:20:00,0.01796,0.01798,0.01791,0.01791,3897690.0,0.018214,0.017762,0.017988,0.0,0,0,0,0.000091,48.558813
2023-08-01 02:25:00,0.01791,0.01796,0.01790,0.01794,3066078.0,0.017975,0.017893,0.017934,0.0,0,0,0,0.000089,51.489473
2023-08-01 02:30:00,0.01793,0.01797,0.01790,0.01792,4510482.0,0.017967,0.017889,0.017928,0.0,0,0,0,0.000088,53.778292
2023-08-01 02:35:00,0.01793,0.01798,0.01793,0.01798,2124079.0,0.017993,0.017891,0.017942,0.0,0,0,0,0.000086,55.517775


In [8]:
#rename columns
df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)

In [9]:
df['Signal'].value_counts()

0    21050
2     2414
1        9
Name: Signal, dtype: int64

In [10]:
from backtesting import Strategy
from backtesting import Backtest

def SIGNAL(df,col_name):
    return df[col_name]

class Scalping_Strategy(Strategy):
    atr_sl_rate = 1.3
    TPSL_rate = 1.5
    save_size_rate = 0.125
    init_cash = 350
    profit_target = 500
    size_multiplier_rate = 0.5

    def init(self):
        super().init()
        self.signal = self.I(SIGNAL, self.data, 'Signal')

    def next(self):
        super().next()

        slatr = self.atr_sl_rate * self.data.ATR[-1]
        size_multiplier = self.size_multiplier_rate if self.equity >= self.profit_target else 1.0

        if self.signal == 2:
            sl_price = self.data.Close[-1] - slatr
            tp_price = self.data.Close[-1] + self.TPSL_rate * slatr
            size = min(self.save_size_rate * self.equity / slatr, self.init_cash / slatr) * size_multiplier
            if self.equity > self.init_cash * 1.5:
                self.buy(sl=sl_price, tp=tp_price, size=int(size))
            else:
                self.buy(sl=sl_price, tp=tp_price)

        if self.signal == 1:
            sl_price = self.data.Close[-1] + slatr
            tp_price = self.data.Close[-1] - self.TPSL_rate * slatr
            size = min(self.save_size_rate * self.equity / slatr, self.init_cash / slatr) * size_multiplier
            if self.equity >= self.init_cash * 1.5:
                self.sell(sl=sl_price, tp=tp_price, size=int(size))
            else:
                self.sell(sl=sl_price, tp=tp_price)
            

bt = Backtest(df, Scalping_Strategy, cash=350, commission=.00002, margin=.05)
stat = bt.run()
stat

Start                     2023-08-01 02:15:00
End                       2023-10-21 14:15:00
Duration                     81 days 12:00:00
Exposure Time [%]                    49.19695
Equity Final [$]                    34.428579
Equity Peak [$]                    798.375485
Return [%]                         -90.163263
Buy & Hold Return [%]              -24.443207
Return (Ann.) [%]                  -99.982616
Volatility (Ann.) [%]     22400939997905.6...
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -98.306961
Avg. Drawdown [%]                  -17.589539
Max. Drawdown Duration       45 days 09:40:00
Avg. Drawdown Duration        5 days 09:43:00
# Trades                                 2121
Win Rate [%]                        39.085337
Best Trade [%]                       2.572146
Worst Trade [%]                     -1.306412
Avg. Trade [%]                    

In [11]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter(x=stat['_equity_curve'].index, y=stat['_equity_curve']['Equity'])])
fig.show()

In [12]:
%%time
stats = bt.optimize(atr_sl_rate = np.arange(0.5, 2.6, 0.2).tolist(),
                    TPSL_rate = np.arange(0.5, 2.6, 0.2).tolist(),
                    size_multiplier_rate = np.arange(0.1, 0.5, 0.1).tolist(),
                    maximize='Equity Final [$]')
stats

C:\Users\hiban\AppData\Roaming\Python\Python310\site-packages\backtesting\backtesting.py:1488: UserWarning:

Searching for best of 484 configurations.



  0%|          | 0/13 [00:00<?, ?it/s]

CPU times: total: 4min 7s
Wall time: 6min 19s


Start                     2023-08-01 02:15:00
End                       2023-10-21 14:15:00
Duration                     81 days 12:00:00
Exposure Time [%]                    27.88736
Equity Final [$]                  5767.346352
Equity Peak [$]                   5909.149234
Return [%]                        1547.813244
Buy & Hold Return [%]              -24.443207
Return (Ann.) [%]             36357019.248868
Volatility (Ann.) [%]      57815544858.776978
Sharpe Ratio                         0.000629
Sortino Ratio                   187431.639399
Calmar Ratio                     489509.45208
Max. Drawdown [%]                  -74.272354
Avg. Drawdown [%]                   -7.579318
Max. Drawdown Duration       22 days 11:00:00
Avg. Drawdown Duration        0 days 19:12:00
# Trades                                 2196
Win Rate [%]                         28.59745
Best Trade [%]                       1.374426
Worst Trade [%]                     -1.013953
Avg. Trade [%]                    

In [13]:
stats.keys()

Index(['Start', 'End', 'Duration', 'Exposure Time [%]', 'Equity Final [$]',
       'Equity Peak [$]', 'Return [%]', 'Buy & Hold Return [%]',
       'Return (Ann.) [%]', 'Volatility (Ann.) [%]', 'Sharpe Ratio',
       'Sortino Ratio', 'Calmar Ratio', 'Max. Drawdown [%]',
       'Avg. Drawdown [%]', 'Max. Drawdown Duration', 'Avg. Drawdown Duration',
       '# Trades', 'Win Rate [%]', 'Best Trade [%]', 'Worst Trade [%]',
       'Avg. Trade [%]', 'Max. Trade Duration', 'Avg. Trade Duration',
       'Profit Factor', 'Expectancy [%]', 'SQN', '_strategy', '_equity_curve',
       '_trades'],
      dtype='object')

In [14]:
stats._strategy

<Strategy Scalping_Strategy(atr_sl_rate=0.7,TPSL_rate=2.3,size_multiplier_rate=0.2)>

In [16]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter(x=stats['_equity_curve'].index, y=stats['_equity_curve']['Equity'])])
fig.update_layout(title='Profit', yaxis_title='USDT', xaxis_title='Date')
fig.show()